In [ ]:
import gym
import numpy as np
import random
from collections import defaultdict
import time

# ------------------------------
# 1. Discretization for Continuous States
# ------------------------------
# Since CartPole’s state (cart position, cart velocity, pole angle, pole velocity) is continuous,
# we create bins for each dimension and discretize the state.
class Discretizer:
    def __init__(self, bins):
        self.bins = bins  # bins: a list of bin arrays for each state dimension

    def discretize(self, state):
        # For each dimension, return the index of the bin where the state value falls.
        discretized = []
        for i, s in enumerate(state):
            discretized.append(np.digitize(s, self.bins[i]))
        return tuple(discretized)

def create_bins(num_bins, lower_bound, upper_bound):
    # Create (num_bins - 1) boundaries evenly spaced between lower_bound and upper_bound.
    return np.linspace(lower_bound, upper_bound, num_bins - 1)

# Define bins for each of the 4 state variables.
cart_position_bins = create_bins(10, -4.8, 4.8)
cart_velocity_bins = create_bins(10, -5, 5)       # rough bounds
pole_angle_bins = create_bins(10, -0.418, 0.418)    # approximately ±24 degrees in radians
pole_velocity_bins = create_bins(10, -5, 5)         # rough bounds
bins = [cart_position_bins, cart_velocity_bins, pole_angle_bins, pole_velocity_bins]

discretizer = Discretizer(bins)

# ------------------------------
# 2. Defining a Random Policy
# ------------------------------
# This function returns a random action (0 or 1) regardless of the state.
def random_policy(state):
    return random.choice([0, 1])

# ------------------------------
# 3. Monte Carlo Policy Evaluation (Episodic Task)
# ------------------------------
# For each episode, we generate a trajectory following the given policy (here, random_policy).
# Then, we compute the discounted return (G) for each state (first-visit MC) and average these to form a value estimate.
def monte_carlo_policy_evaluation(env, policy, num_episodes=1000, gamma=0.99):
    returns_sum = defaultdict(float)   
    returns_count = defaultdict(float)   
    value_function = defaultdict(float)
    
    for i_episode in range(1, num_episodes + 1):
        episode = []
        state = env.reset()
        state = discretizer.discretize(state)
        done = False

        while not done:
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            state = discretizer.discretize(next_state)
        
        G = 0
        for t in reversed(range(len(episode))):
            state_t, action_t, reward_t = episode[t]
            G = gamma * G + reward_t
            if state_t not in [x[0] for x in episode[:t]]:
                returns_sum[state_t] += G
                returns_count[state_t] += 1.0
                value_function[state_t] = returns_sum[state_t] / returns_count[state_t]
    
    return value_function

# ------------------------------
# 4. Q-Learning for Approximating the Optimal Policy and Value Function
# ------------------------------
def q_learning(env, num_episodes=5000, alpha=0.1, gamma=0.99, epsilon=0.1):
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    for i_episode in range(1, num_episodes + 1):
        state = env.reset()
        state = discretizer.discretize(state)
        done = False
        
        while not done:
            if random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, done, _ = env.step(action)
            next_state = discretizer.discretize(next_state)
            best_next_action = np.argmax(Q[next_state])
            td_target = reward + gamma * Q[next_state][best_next_action] * (not done)
            td_error = td_target - Q[state][action]
            Q[state][action] += alpha * td_error
            state = next_state

    optimal_policy = {}
    optimal_value_function = {}
    for state, actions in Q.items():
        optimal_policy[state] = np.argmax(actions)
        optimal_value_function[state] = np.max(actions)
    
    return Q, optimal_policy, optimal_value_function

# ------------------------------
# 5. Main Routine: Running the Experiments
# ------------------------------
def main():
    env = gym.make('CartPole-v1')
    
    print("Evaluating Random Policy with Monte Carlo Policy Evaluation...")
    random_value_function = monte_carlo_policy_evaluation(env, random_policy, num_episodes=1000)
    print("Random Policy Value Function (sample of states):")
    for state, value in list(random_value_function.items())[:10]:
        print(f"State: {state}, Estimated Value: {value:.2f}")
    
    print("\nLearning Optimal Policy with Q-Learning...")
    Q, optimal_policy, optimal_value_function = q_learning(env, num_episodes=5000)
    print("Optimal Policy and Value Function (sample of states):")
    for state, value in list(optimal_value_function.items())[:10]:
        print(f"State: {state}, Optimal Value: {value:.2f}, Optimal Action: {optimal_policy[state]}")
    
    print("\nVisualizing Random Policy on CartPole Environment...")
    for episode in range(5):
        state = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            env.render()
            action = random_policy(state)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            time.sleep(0.02)
        
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")
    
    env.close()

if __name__ == '__main__':
    main()
